# VGFFMC Class-less


In [14]:
from numpy.linalg import norm, solve
import numpy as np
from matplotlib import pyplot as plt
from random import uniform
from math import pi
import numpy as np
import cmath as cm
from collections import Counter
import json
import time
from numpy.linalg import norm
from numpy import arange,array,pi,cos,sin,copy,sqrt
from matplotlib import pyplot as plt
    
    #def __init__(self,mcount,kth,kph,NK,divd,khatN,alpha,beta):       
        #self.theta = theta                    
        #self.phi = phi
        #self.mcount = mcount
        #self.kth = kth
        #self.kph = kph
        #self.NK = NK
        #self.divd = divd
        #self.khatN = khatN
        #self.alpha = alpha   # only adding to test the thetas and phis 
        #self.beta = beta     # that arnt in an array
    
def vectors(alpha,beta):
    khat = np.zeros(3)
    #khat[0] = np.sin(theta)*np.cos(phi)
    #khat[1] = np.sin(theta)*np.sin(phi)
    #khat[2] = np.cos(theta)
        
    khat[0] = np.sin(alpha)*np.cos(beta)
    khat[1] = np.sin(alpha)*np.sin(beta)
    khat[2] = np.cos(alpha)
    return khat
    
def vectors3(mcount,kth,kph,NK,divd,khatN,alpha,beta):                      
    #khatN = self.khatN
    #kth = self.kth
    #kph = self.kph
        
    for N in range(NK):          
        khatN[N][0]=np.sin(kth[N])*np.cos(kph[N]) 
        khatN[N][1]=np.sin(kth[N])*np.sin(kph[N])
        khatN[N][2]=np.cos(kth[N])
    return khatN
    
def move1KV(mcount,kth,kph,NK,divd,khatN,alpha,beta):
    THinc = np.pi/divd                 
    PHinc = 2*THinc                    
    temp1 = THinc*uniform(-1,1) 
    temp2 = PHinc*uniform(-1,1)
        
    for i in range(NK):
            
        kth[i] = kth[i]+ temp1
        kph[i] = kph[i] + temp2 
        
        if (kth[i] > np.pi):
            kth[i] = 2*np.pi-kth[i]
        if (kph[i] > np.pi*2):
            kph[i] = kph[i]-np.pi*2
        if (kth[i] < 0.0):
            kth[i] = -kth[i]
        if (kph[i] < 0.0):
            kph[i] = -kph[i]
         
        khatN[i]= vectors(kth[i], kph[i])
        
    return kth, kph, khatN 

        
def ROT(alpha,beta,gamma):
    RRR = np.array(
      [[ cos(alpha)*cos(beta)*cos(gamma)-sin(alpha)*sin(gamma),
         sin(alpha)*cos(beta)*cos(gamma)+cos(alpha)*sin(gamma),
        -sin(beta)*cos(gamma)],
       [-cos(alpha)*cos(beta)*sin(gamma)-sin(alpha)*cos(gamma),
        -sin(alpha)*cos(beta)*sin(gamma)+cos(alpha)*cos(gamma),             
         sin(beta)*sin(gamma)],
        [cos(alpha)*sin(beta),
         sin(alpha)*sin(beta),
         cos(beta)]])
    return RRR

def dd(alpha,beta,i,j):
    dd=0.0
    if ((alpha == beta) and (i == j)):
        dd=1.0
    return dd
    
def delta(alpha,beta,i,j):
    d=0.0
    if (alpha == beta ):
        d=1.0
    return d

def GAM(R,dtemp,k,eps,a,i,b,j):
    GAM=0.0+0.0j
    kd=k*dtemp
    b1=(3./(4.*np.pi))**(2./3.)
    GAM=b1*(kd)**2 + 1j*kd**3/(2.*np.pi)          
    return GAM
    
def GG(R,dtemp,k,eps,a,i,b,j):
        
    PHZ = 0.0+0.0j
    t1 = 0.0+0.0j
    t2 = 0.0+0.0j
    temp = 0.0+0.0j
    ci = 0.0+1.0j

    K2= k * k 
    
        
    Rab = np.zeros(3)
    
    Rhat = np.zeros(3)
    if not b == a:
            
            
        Rab[0] = R[a][0] - R[b][0]
        Rab[1] = R[a][1] - R[b][1]
        Rab[2] = R[a][2] - R[b][2]
            
        RMAG = Rab[0]**2 + Rab[1]**2 +Rab[2]**2 
        RMAG = RMAG**0.5
        Rhat[0] = Rab[0]/RMAG
        Rhat[1] = Rab[1]/RMAG
        Rhat[2] = Rab[2]/RMAG
            
        temp = ci*k*RMAG
        PHZ = np.exp(temp)
        t1 = (K2/RMAG)*(delta(alpha,beta,i,j) - Rhat[i]*Rhat[j])
        t2 = (ci*k/RMAG**2 - 1.0/RMAG**3) * (delta(alpha,beta,i,j)-3.0*Rhat[i]*Rhat[j])
        G = PHZ * (t1 + t2)
    else:    
        G = 4.0*np.pi*GAM(R,dtemp,k,eps,a,i,b,j)/(3.0*dtemp**3)
    return G

def CPSI(R,khatN, k, mm,N,b) :
    KDR=khatN[N].dot(R[b])
    temp = 1j*mm*k*KDR # use complex notation maybe?
    return np.exp(temp)

def ecalc(R,khatN,k,mm,NUSE,An,x,NK, E) :
        
    F = np.zeros((NUSE, 3),dtype=complex)
    for b in range(0,NUSE):
        for l in range(3):
            F[b][l]=(0.0)
            for N in range(0,NK):
                F[b][l]=F[b][l]+ An[N][l]*CPSI(R,khatN, k, mm, N, b)          
     
        #E = np.zeros((NUSE,3),dtype=complex)
    for m in range(0,NUSE):
        for I in range(3):
            E[m][I]=F[m][I]/(1.+(4.*np.pi)* x/3.)
        #
    return E

def Calculate_T1_Matrix(D, R, k, khatN, eps, mm, w, NUSE, NK,T1):
    for a in range(NUSE):
        for i in range(3):
            for N in range (NK):
                for j in range(3):
                    T1[a][i][N][j] = 0.0+0.0j
                    for b in range (NUSE):
                        dtemp = D[b]
                        #dirac = DoubleDirac(a,b,i,j)
                        #greens = Greens(R,d,k,eps,a,b,i,j,dirac)
                        #efield = Efield(R,khatN,k,mm,NUSE,An,x,NK, E)
                        #print(greens.GG)
                        #print(dirac.dd(),a,i,b,j)
                        #print(a,i,N,j, T1)
                        T1[a][i][N][j] =  T1[a][i][N][j] + (dd(a,i,b,j)- D[b]**3 * w * GG(R,dtemp,k,eps,a,i,b,j) * CPSI(R,khatN, k, mm, N, b))
                    #print(a,i,N,j, T1)        
    return T1

def Calculate_Y_Matrix(T1, E0, NUSE, NK):
    print('Building the Y matricx')
    for M in range(NK):
        for l in range(3):
            Y[M][l] = 0.0 + 0.0j    
            for a in range(NUSE):
                for i in range(3):
                    #T = T_Matrix(D, R, k, khatN, eps, mm, w, NUSE, NK,dirac,greens,efield,T1)
                    #T1 = T.Calculate_T1_Matrix()
                    Y[M][l] = Y[M][l] + np.conjugate(T1[a][i][M][l])*E0[a][i]
    return Y

def Calculate_H_Matrix(T1, NK, NUSE):
    print('Building the H matrix')
    for M in range (NK):
        for l in range (3):
            for N in range(NK):
                for j in range(3):
                    H[M][l][N][j] = complex(0,0)
                    for a in range(NUSE):
                        for i in range(3):
                            #T = T_Matrix(D, R, k, khatN, eps, mm, w, NUSE, NK,dirac,greens,efield,T1)
                            #T1 = T.Calculate_T1_Matrix()
                            H[M][l][N][j] =  H[M][l][N][j] + np.conjugate(T1[a][i][M][l]) * (T1[a][i][N][j])
    return H 

def Reform_Y_H_Matricies(Y, H, NK, bb, aa):
    print('reforming the matrix for solving ')
    for n in range(NK):
        for i in range(3):
            m = 3*(n)+i 
            bb[m] = Y[n][i]
                
            for l in range (NK):
                for j in range (3):
                    p = 3*(l)+j
                    aa[m][p] = H[n][i][l][j]
                        
                    #j += 1
                #l += 1
            #i += 1
        #n += 1
            
    return aa, bb

def Calculate_Phi(An, H, Y, E0, NK, NUSE):
    CPHI=0.0+0.0j
    for N in range(0,NK):
        for j in range(0,3):
            for M in range(0,NK):
                for l in range (0,3):
                    CPHI = CPHI + np.conjugate(An[M][l]*H[M][l][N][j]*An[N][j])
             
    for j in range (3):
        for N in range (0, NK):
            CPHI = CPHI - (np.conjugate(Y[N][j])*An[N][j]                     \
                       +Y[N][j]*np.conjugate(An[N][j]))
      
    for a in range(NUSE):
        for i in range(3):
            CPHI - CPHI + E0[a][i]*np.conjugate(E0[a][i])       
    return CPHI

startTime=time.time()

DEG = pi/180.0
CI = (0.0,1.0)
nmax = 20000
knmax = 150
mmax = 10
ERR0 = 0.05
jcount = 1
divd = 100
d1 = 0

#VGFIN = input("Enter vgfin input file name: ")
#VGFKV = input("Enter vgfkv input file name: ")
#OUTFILE = input("Enter output file name: ")

VGFIN = "yo"
VGFKV = "mama"
OUTFILE = "hi"

with open("/Users/madisonderosier/Desktop/Research/" + VGFIN + ".txt",'r')as fp:    # Read in VGFIN file
    variable=fp.read()
    vgfin=json.loads(variable)


with open("/Users/madisonderosier/Desktop/Research/" + VGFKV + ".txt",'r')as fp:   # Read in VGFKV
    variable=fp.read()
    vgfkv=json.loads(variable)

wave = int(vgfin["Wavelength"])                    # This just reads in each individual variable and
alpha = int(vgfin["Incident theta angle"])         #     asigns them the variable that will be used
beta = int(vgfin["Value for phi"])                 #     to call them in the functions in this code
gamma = int(vgfin["Value for gamma"])
psi = int(vgfin["Polarization angle"])
RAD = int(vgfin["Particle symmetry semi-axes"])
eps = complex(vgfin["EPS"])
er = float(vgfin["ER"])
ei = float(vgfin["EI"])
td = float(vgfin["TD"])
R = np.array(vgfin["R-vector"])
D = np.array(vgfin["D-vector"])
nsid = int(vgfin["Number of dipoles along major axes"])
nlsid = int(vgfin["Number of fine structure devisions per dipole cell side"])
mr = float(vgfin["Real part of index of refraction"])
mi = float(vgfin["Imaginary part of index of refraction"])
NUSE = int(vgfin["Counter"])
print(D)

kth = np.array(vgfkv["Theta angle"])
kph = np.array(vgfkv["Phi angle"])
lastLine = np.array(vgfkv["Last Line"])
NK = vgfkv["Count"]
mcount = np.array(vgfkv["CountList"])

ERR = lastLine[0]
ERRlast = lastLine[1]
kcount = lastLine[2]

khat = np.zeros(3)

EPS = complex(er,ei)
mm = np.sqrt(EPS)
x = complex(0,0)
x = (EPS - 1.0)/(4.0* pi)
temp = complex(0,0)
C = complex(0,0)
w = x/(1.0 + (4.0 * np.pi / 3.0) * x)

khatN = np.zeros((NK,3))

#print(R)
#print(R[0][0])
 
d2 = []
for i in range(1,NUSE): 
                                        # This is supposed to be the loop that calls in each 
    R1 = R[i][0]  
    R2 = R[i][1]
    R3 = R[i][2]                        #     set of (x,y, and z) components of R and D in order
    dsum = D[i]                   #     to calculate the total volume of the particle 
    d1 = d1 + dsum**3 
    d2.append(d1)
                               # Its currently commented out because it was returning ani+= 1                       #     error saying there were too many values in the R array?
                                 #     so I was very confused but wanted to make sure the rest of the code worked.
print(dsum)

dtemp = d2[-1]
k = (2.0 * pi)/ wave            # Defines k
alpha = alpha*DEG               # The following makes the following variables in terms of radians
beta = beta*DEG
gamma = gamma*DEG
psi = psi*DEG
#theta = theta*DEG
#phi = phi*DEG

#R = np.zeros((NUSE,3))  
#D = np.zeros((NUSE))
An = np.zeros((NK,3),dtype=complex) 
T1 = np.zeros((NUSE,3,NK,3),dtype=complex)
E = np.zeros((NUSE,3),dtype=complex)
E0 = np.zeros((NUSE,3),dtype=complex)
F = np.zeros((NUSE,3),dtype=complex)
Y = np.zeros((NK,3),dtype=complex)
H = np.zeros((NK, 3, NK,3),dtype=complex)
aa = np.zeros((3*NK,3*NK),dtype=complex)
bb = np.zeros((NK*3),dtype=complex)
xx = np.zeros((3*NK), dtype = complex)


                     
khat = vectors(alpha,beta)
kvectors = vectors3(mcount,kth,kph,NK,divd, khatN,alpha,beta)
kth,kph,khatN = move1KV(mcount,kth,kph,NK,divd, khatN,alpha,beta)

    # This is where Class ROT is called in and RRR can be 
RRR = ROT(alpha,beta,gamma)                 # Successfully executed!

                     
#ddelta = dd(alpha,beta,i,j)
#delta = delta(alpha,beta,i,j)


#GAM = GAM(R,dtemp,k,EPS,alpha,i,beta,j)
#G = GG(R,dtemp, k, eps, alpha, i, beta, j)

       # Do I need to include N and b?               
#E = ecalc(R,khatN,k,mm,NUSE,An,x,NK,E)


v = np.zeros((3))
v[0] = 0.0
v[1] = 0.0
v[2] = 1.0
khat = RRR.dot(v)

v[0] = np.cos(psi)
v[1] = np.sin(psi)
v[2] = 0.0
E0hat = RRR.dot(v)

for i in range(0,NUSE):
    RDK = np.dot(khat,R[i])
    temp = 1j * k * RDK
    C = np.exp(temp)
    E0[i] = C * E0hat
    
# Monte Carlo Loop (I Believe)
#print(ERR, ERR0, jcount, mmax)
while (ERR > ERR0) and (jcount < mmax):
    print("inside loop", ERR, ERR0, jcount, mmax)
    for kcount in range(1):
        print(kcount)
        
        T1 = Calculate_T1_Matrix(D,R,k,khatN,eps,mm,w,NUSE,NK,T1)
        print(T1)
        
        
        Y = Calculate_Y_Matrix(T1,E0,NUSE,NK)
        print(Y)
        
        
        H = Calculate_H_Matrix(T1,NK, NUSE)
        print(H)
        
        
        aa, bb = Reform_Y_H_Matricies(Y,H,NK,bb,aa)
        print(aa,bb)
        
        xx = np.linalg.solve(aa,bb)
        print(xx)
        
        for N in range(NK):
            for i in range(3):
                M = 3*(N-0)+i
                An[N][i] = xx[M]
        
        
        Phi = Calculate_Phi(An, H, Y, E0, NK, NUSE)
        print(Phi)
        
        ERR = Phi * np.conjugate(Phi)
        
        if (ERR>ERR0):
            if (ERR > ERRlast):
                ERRlast = ERR
                khatN = vectors3(mcount,kth,kph,NK,divd, khatN,alpha,beta)
                
            else: 
                
                ERRlast = ERR
                E = ecalc(R,khatN,k,mm,NUSE,An,x,NK,E)
                
                if(kcount < NK):
                    kth, kph, khatN = move1KV(mcount,kth,kph,NK,divd, khatN,alpha,beta)
        
        else:
            
            E = ecalc(R,khatN,k,mm,NUSE,An,x,NK,E)
            break
            
    print('End of kcount loop')
    
    jcount += 1
    print('End Monte Carlo Loop, mcount:', jcount,'ERR = ', ERR)
    
print('Print out the E-vectors')
print(E)
print('NUSE', NUSE)

print(time.time()-startTime())

[[0.21203702]
 [0.3058103 ]
 [0.21203702]
 [0.21203702]
 [0.49856999]
 [0.56180121]
 [0.49856999]
 [0.21203702]
 [0.3058103 ]
 [0.56180121]
 [0.62      ]
 [0.56180121]
 [0.3058103 ]
 [0.21203702]
 [0.49856999]
 [0.56180121]
 [0.49856999]
 [0.21203702]
 [0.21203702]
 [0.3058103 ]
 [0.21203702]
 [0.21203702]
 [0.49856999]
 [0.56180121]
 [0.49856999]
 [0.21203702]
 [0.49856999]
 [0.62      ]
 [0.62      ]
 [0.62      ]
 [0.49856999]
 [0.56180121]
 [0.62      ]
 [0.62      ]
 [0.62      ]
 [0.56180121]
 [0.49856999]
 [0.62      ]
 [0.62      ]
 [0.62      ]
 [0.49856999]
 [0.21203702]
 [0.49856999]
 [0.56180121]
 [0.49856999]
 [0.21203702]
 [0.3058103 ]
 [0.56180121]
 [0.62      ]
 [0.56180121]
 [0.3058103 ]
 [0.56180121]
 [0.62      ]
 [0.62      ]
 [0.62      ]
 [0.56180121]
 [0.62      ]
 [0.62      ]
 [0.62      ]
 [0.62      ]
 [0.62      ]
 [0.56180121]
 [0.62      ]
 [0.62      ]
 [0.62      ]
 [0.56180121]
 [0.3058103 ]
 [0.56180121]
 [0.62      ]
 [0.56180121]
 [0.3058103 ]
 [0.21

[[[[15.95787167+0.j          6.92605938-0.03739256j
     7.60606237-0.0202344j ]
   [15.40168149-0.02235123j  7.06625185-0.06623705j
     7.62976366-0.01085675j]
   [15.45764348+0.04628763j  6.73005217+0.01103271j
     7.32858413+0.02782454j]
   [15.95423191-0.07799775j  7.1119519 -0.06559789j
     7.39085773-0.07770952j]
   [14.59424812-0.03985099j  7.11558389-0.10590947j
     7.30439317-0.01890771j]
   [14.7215247 +0.06789706j  6.80998388+0.05208092j
     7.04786721+0.07620778j]
   [15.15514642-0.16223366j  7.18361201-0.08144134j
     7.07348224-0.11040089j]
   [14.00923232-0.04484267j  7.06975406-0.10098571j
     6.97956483-0.01885045j]
   [14.08203324+0.03791242j  6.97005054+0.04527922j
     6.89756502+0.07899598j]]

  [[ 6.92605938+0.03739256j 15.8903882 +0.j
     7.58312593+0.01715498j]
   [ 6.9983749 +0.01343865j 15.7881334 -0.04903624j
     7.26615362+0.00140978j]
   [ 6.72863972+0.08362298j 15.29106555+0.05011591j
     7.41898891+0.072782j  ]
   [ 7.16702051-0.02740467j 15.564

[[[[ 0.95414246-3.54504919e-02j  1.05500013-4.22631455e-02j
     1.09311434-7.27156248e-02j]
   [ 0.954673  -1.87630343e-03j  1.05342396-4.70316589e-02j
     1.08895723-5.76603528e-02j]
   [ 0.96527151-7.20381306e-02j  1.04516252-1.00221985e-02j
     1.09563637-3.17625970e-02j]
   ...
   [ 0.95233862-7.60214569e-02j  1.04816696-7.75979415e-02j
     1.08311499-7.20393948e-02j]
   [ 0.95173254-4.01503368e-02j  1.05718149-4.26820967e-02j
     1.08165622+2.91571965e-03j]
   [ 0.99457267-9.81163666e-02j  1.03862646-1.03857759e-02j
     1.06106127+1.60336255e-02j]]

  [[ 0.05500013-4.22631455e-02j -0.04590068-3.43021366e-02j
     0.09297487-7.24933243e-02j]
   [ 0.05342396-4.70316589e-02j -0.04151723-7.70931838e-02j
     0.09584754-7.27372435e-02j]
   [ 0.04516252-1.00221985e-02j -0.04674044-4.86243496e-02j
     0.09106067-2.80556574e-02j]
   ...
   [ 0.04816696-7.75979415e-02j -0.03864254-7.73618840e-03j
     0.08553686-5.73252061e-02j]
   [ 0.05718149-4.26820967e-02j -0.01738023-1.11718478

[[[[15.96209947+0.j          6.92871579-0.03729355j
     7.60702752-0.02014001j]
   [15.41529118-0.01797759j  7.05258316-0.06196057j
     7.63586996-0.00705065j]
   [15.49322303+0.04378725j  6.73467991+0.00912922j
     7.33229105+0.02467131j]
   [15.97885529-0.0751156j   7.11666349-0.06634227j
     7.41360639-0.07669352j]
   [14.60597562-0.03408555j  7.10714699-0.10013172j
     7.3185833 -0.0120058j ]
   [14.75504931+0.06450816j  6.80728633+0.05108055j
     7.04537942+0.07326544j]
   [15.20308574-0.1599864j   7.19183899-0.08502322j
     7.09428565-0.11301053j]
   [14.01719838-0.03929176j  7.06779847-0.09824093j
     6.98865449-0.01369381j]
   [14.10122166+0.03719803j  6.96723669+0.04774402j
     6.89575642+0.07935918j]]

  [[ 6.92871579+0.03729355j 15.8913888 +0.j
     7.58508221+0.01714959j]
   [ 6.98279355+0.01787936j 15.79602085-0.04410084j
     7.27424863+0.00486602j]
   [ 6.73731698+0.08160081j 15.30574977+0.04789832j
     7.43857946+0.07062502j]
   [ 7.1688063 -0.02614875j 15.610

[[[[ 0.95414246-3.54504919e-02j  1.05500013-4.22631455e-02j
     1.09311434-7.27156248e-02j]
   [ 0.954673  -1.87630343e-03j  1.05342396-4.70316589e-02j
     1.08895723-5.76603528e-02j]
   [ 0.96527151-7.20381306e-02j  1.04516252-1.00221985e-02j
     1.09563637-3.17625970e-02j]
   ...
   [ 0.95233862-7.60214569e-02j  1.04816696-7.75979415e-02j
     1.08311499-7.20393948e-02j]
   [ 0.95173254-4.01503368e-02j  1.05718149-4.26820967e-02j
     1.08165622+2.91571965e-03j]
   [ 0.99457267-9.81163666e-02j  1.03862646-1.03857759e-02j
     1.06106127+1.60336255e-02j]]

  [[ 0.05500013-4.22631455e-02j -0.04590068-3.43021366e-02j
     0.09297487-7.24933243e-02j]
   [ 0.05342396-4.70316589e-02j -0.04151723-7.70931838e-02j
     0.09584754-7.27372435e-02j]
   [ 0.04516252-1.00221985e-02j -0.04674044-4.86243496e-02j
     0.09106067-2.80556574e-02j]
   ...
   [ 0.04816696-7.75979415e-02j -0.03864254-7.73618840e-03j
     0.08553686-5.73252061e-02j]
   [ 0.05718149-4.26820967e-02j -0.01738023-1.11718478

...

[[[[ 0.95418002-0.03572929j  1.05498645-0.04256364j
     1.09306102-0.07306364j]
   [ 0.95598864-0.00090333j  1.05314802-0.05013749j
     1.08832495-0.06021636j]
   [ 0.96431645-0.07005008j  1.0444953 -0.0089797j
     1.09470171-0.02999427j]
   ...
   [ 0.95349277-0.08151011j  1.04935918-0.07469231j
     1.08456008-0.06977057j]
   [ 0.9520654 -0.03962133j  1.05784047-0.04553472j
     1.08285727+0.00055769j]
   [ 0.9926325 -0.09617952j  1.03819843-0.00998292j
     1.06002376+0.01710092j]]

  [[ 0.05498645-0.04256364j -0.04588898-0.03372318j
     0.09281878-0.07267453j]
   [ 0.05314802-0.05013749j -0.04265408-0.07531218j
     0.09454388-0.07535372j]
   [ 0.0444953 -0.0089797j  -0.04512872-0.0523108j
     0.09116949-0.027209j  ]
   ...
   [ 0.04935918-0.07469231j -0.03923836-0.0056913j
     0.08668066-0.05344818j]
   [ 0.05784047-0.04553472j -0.02005102-0.1101401j
     0.0830201 -0.01666231j]
   [ 0.03819843-0.00998292j -0.02191257-0.08328935j
     0.05781539+0.02091628j]]

  [[ 0.0930610

[[[[15.97226036+0.00000000e+00j  6.93322048-3.70888178e-02j
     7.60734452-2.00429038e-02j]
   [15.43464811-2.28513153e-02j  7.07383366-6.59275879e-02j
     7.63645724-1.18888971e-02j]
   [15.46378614+4.74683816e-02j  6.73044964+1.12016444e-02j
     7.34150384+2.92612413e-02j]
   [15.97866682-7.40852179e-02j  7.10941603-6.30866077e-02j
     7.39765234-7.46209225e-02j]
   [14.62305534-4.23237945e-02j  7.12419407-1.06931020e-01j
     7.3126825 -2.16650896e-02j]
   [14.72587323+7.08460302e-02j  6.80143486+5.26633247e-02j
     7.05614261+7.81823434e-02j]
   [15.19203917-1.59510556e-01j  7.18661315-7.79568439e-02j
     7.07951525-1.07036271e-01j]
   [14.02105257-4.70978705e-02j  7.0739575 -1.03171068e-01j
     6.9827385 -2.13354780e-02j]
   [14.08492828+4.28899597e-02j  6.96307945+4.73345128e-02j
     6.899038  +8.23507124e-02j]]

  [[ 6.93322048+3.70888178e-02j 15.8897768 +0.00000000e+00j
     7.58941708+1.70390177e-02j]
   [ 7.00347436+1.19337505e-02j 15.80513108-4.86593082e-02j
     7.2

[[[[ 0.95417918-0.03565447j  1.05498558-0.04258544j
     1.09304558-0.07307172j]
   [ 0.95532964-0.00124148j  1.05329175-0.04861082j
     1.08864929-0.05879564j]
   [ 0.9649453 -0.07124645j  1.04471734-0.00935826j
     1.09509769-0.03061551j]
   ...
   [ 0.95288558-0.07890127j  1.04880561-0.07617532j
     1.0838522 -0.07076825j]
   [ 0.95194789-0.04010014j  1.05754652-0.04406784j
     1.08221849+0.00185461j]
   [ 0.99357573-0.09714804j  1.03848619-0.01024212j
     1.06051851+0.01661353j]]

  [[ 0.05498558-0.04258544j -0.04588502-0.03377489j
     0.09281871-0.07270709j]
   [ 0.05329175-0.04861082j -0.04204899-0.07646961j
     0.095219  -0.07399087j]
   [ 0.04471734-0.00935826j -0.04592097-0.05059161j
     0.09101394-0.0273606j ]
   ...
   [ 0.04880561-0.07617532j -0.03894673-0.00676479j
     0.08616484-0.05523414j]
   [ 0.05754652-0.04406784j -0.01870487-0.1108993j
     0.08269738-0.01553205j]
   [ 0.03848619-0.01024212j -0.02327167-0.08207869j
     0.05797234+0.02106255j]]

  [[ 0.0930

[[[[15.97180146+0.00000000e+00j  6.93354727-3.70890684e-02j
     7.60788475-2.00125761e-02j]
   [15.42128776-2.02456171e-02j  7.06515673-6.38328344e-02j
     7.6357059 -8.99823399e-03j]
   [15.47407949+4.61916992e-02j  6.73225402+1.07876440e-02j
     7.33483446+2.77632260e-02j]
   [15.98222755-7.48497995e-02j  7.11643398-6.46885081e-02j
     7.40599292-7.56894526e-02j]
   [14.60970117-3.79309493e-02j  7.1163621 -1.03370386e-01j
     7.3133546 -1.64098320e-02j]
   [14.733906  +6.81854632e-02j  6.8042935 +5.24812712e-02j
     7.04845074+7.65268636e-02j]
   [15.19825016-1.59880517e-01j  7.19161768-8.13810680e-02j
     7.08712588-1.09906894e-01j]
   [14.01604849-4.29265113e-02j  7.07083165-1.00163101e-01j
     6.9837516 -1.70348507e-02j]
   [14.08818586+4.01246645e-02j  6.96497007+4.77585050e-02j
     6.89568057+8.12203309e-02j]]

  [[ 6.93354727+3.70890684e-02j 15.89096753+0.00000000e+00j
     7.58930469+1.70700307e-02j]
   [ 6.9936932 +1.48823603e-02j 15.79684708-4.66558869e-02j
     7.2

TypeError: 'float' object is not callable